------------------
```markdown
# Copyright © 2024 Meysam Goodarzi
This notebook is licensed under CC BY-NC 4.0 with the following amandments:
- Individuals may use, share, and adapt this material for non-commercial purposes with attribution.
- Institutions/Companies must obtain written consent to use this material, except for nonprofits.
- Commercial use is prohibited without permission.  
Contact: analytica@meysam-goodarzi.com
```
------------------------------
❗❗❗ **IMPORTANT**❗❗❗ **Create a copy of this notebook**

In order to work with this Google Colab you need to create a copy of it. Please **DO NOT** provide your answers here. Instead, work on the copy version. To make a copy:

**Click on: File -> save a copy in drive**

Have you successfully created the copy? if yes, there must be a new tab opened in your browser. Now move to the copy and start from there!

----------------------------------------------


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sb
from scipy.stats import weibull_min, lognorm

!pip install quantecon
import quantecon as qe

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.7/322.7 kB 2.8 MB/s eta 0:00:00


# Income Distribution
There are multiple distribution that are normally chosen to represent the real-worl income distributions. Those are:
* Weibull
* Log-normal
* Exponential

## Weibull Distribution
The Weibull distribution is a continuous probability distribution commonly used in reliability analysis, survival studies, and income distributions. It is highly flexible and can model different types of data by adjusting its shape parameter.The probability density function (PDF) of the Weibull distribution is:
$$
f(x; c, \lambda) = \frac{c}{\lambda} \left(\frac{x}{\lambda}\right)^{c-1} e^{-(x/\lambda)^c}, \quad x \geq 0
$$
where:  
- $ x $ is the random variable (income, lifespan, etc.).
-$ c $ (shape parameter) controls the shape of the distribution:
  - If $ c < 1 $,  the distribution is decreasing (high failure rates at the beginning, e.g., early-life failures).
  - If $ c = 1 $ it reduces to an exponential distribution (constant failure rate).
  - If $ c > 1 $ it is bell-shaped, modeling a life cycle with increasing failure rates over time.
- $ \lambda $ (scale parameter) stretches or shrinks the distribution.

In [ ]:
# Weibull distribution parameters
c, loc, scale, population = 1.8, 0, 43.8, 10000  # Shape, location, scale, and population size

# Generate income using Weibull distribution
income = weibull_min.rvs(c=c, loc=loc, scale=scale, size=population)

# Compute the top decile income share (income share of the top 100 earners)
top_dec = np.sum(np.sort(income)[-100:]) / np.sum(income)
print(f"Top decile income share = {top_dec:.2f}")

# Compute median and mean income
median = np.median(income)
mean = np.mean(income)

# Define histogram bin edges (income ranges)
bins = [0] + list(range(1, 150, 1)) + [175, 200, 300, 500]

# Compute histogram of income distribution
hist, bin_edges = np.histogram(income, bins=bins, density=False)
y = [0, max(hist)]  # Y-axis range for reference lines

# Compute Lorenz curve and Gini coefficient
cdf, xcdf = qe.lorenz_curve(income)
gini = qe.gini_coefficient(income)
print(f"Gini Coefficient = {gini:.2f}")

# --------- Plot 1: Income Distribution Histogram ---------
fig, ax = plt.subplots(figsize=(7, 3))

# Bar chart of income distribution
ax.bar(bins[:-1], hist, color="black")

# Vertical lines for mean and median income
ax.plot([median, median], y, color="red", label=f"Median income = {median:.2f}")
ax.plot([mean, mean], y, color="blue", label=f"Mean income = {mean:.2f}")

# Labels and title
ax.set_title("Income Distribution (Weibull dist.)")
ax.set_xlabel("Income (k€)")
ax.set_ylabel("Frequency")
ax.legend()

# --------- Plot 2: Lorenz Curve for Income Inequality ---------
fig, ax = plt.subplots(figsize=(7, 3))

# Lorenz curve: Shows cumulative income distribution
ax.plot(cdf, xcdf, color="black", label="Cumulative income")

# Perfect equality line (45-degree line)
ax.plot(cdf, cdf, color="blue", linestyle="--", label="Perfect equality")

# Labels and legend
ax.set_xlabel("Population portion")
ax.set_ylabel("Income portion")
ax.legend()

# Show both plots
plt.show()


## Log-normal Distribution
The log-normal distribution is a continuous probability distribution of a random variable whose logarithm is normally distributed. It is widely used in economics, finance, and reliability analysis to model variables that are positively skewed, such as income distributions. The probability density function of a log-normal distribution is:

$$
f(x; \mu, \sigma) = \frac{1}{x \sigma \sqrt{2\pi}} e^{-\frac{(\ln x - \mu)^2}{2\sigma^2}}, \quad x > 0
$$

where:  
- $ x $ is the random variable (e.g., income).  
- $ \mu $ location parameter is the mean of the natural logarithm of $ x $  
- $ \sigma $ scale parameter is the standard deviation of the natural logarithm of $ x $  

In [ ]:
# Define parameters for the log-normal distribution
# Standard deviation of log(X)
shape_param = 1.0
# Location parameter (shift)
loc_param = 10
# Scale parameter (median of exp(X))
scale_param = 1.5
population_size = 1000

# Generate income data from a log-normal distribution
income = lognorm.rvs(s=shape_param, loc=loc_param, scale=scale_param, size=population_size)

# Calculate top decile income share (percentage of total income held by the top 10%)
top_decile_income = np.sum(np.sort(income)[-100:]) / np.sum(income)
print(f"Top Decile Income Share: {top_decile_income:.2f}")

# Compute key statistics
median_income = np.median(income)
mean_income = np.mean(income)

# Create histogram bins automatically
hist, bin_edges = np.histogram(income, bins="auto", density=False)
y_range = [0, max(hist)]  # Used for vertical lines indicating median/mean

# Compute Lorenz curve and Gini coefficient
cdf, xcdf = qe.lorenz_curve(income)
gini_coeff = qe.gini_coefficient(income)
print(f"Gini Coefficient: {gini_coeff:.2f}")

# Plot histogram of income distribution
fig, ax = plt.subplots(figsize=(7, 3))
ax.bar(bin_edges[:-1], hist, color="black", alpha=0.7, label="Income Distribution")
ax.axvline(median_income, color="red", linestyle="--", label=f"Median Income = {median_income:.2f}")
ax.axvline(mean_income, color="blue", linestyle="--", label=f"Mean Income = {mean_income:.2f}")

ax.set_title("Income Distribution (Log-Normal)")
ax.set_xlabel("Income (€)")
ax.set_ylabel("Frequency")
ax.legend()

# Plot Lorenz curve for income inequality
fig, ax = plt.subplots(figsize=(7, 3))
ax.plot(cdf, xcdf, color="black", label="Cumulative Income")
ax.plot(cdf, cdf, color="blue", linestyle="--", label="Perfect Equality Line")

ax.set_title("Lorenz Curve")
ax.set_xlabel("Population Portion")
ax.set_ylabel("Income Portion")
ax.legend()

plt.show()


# Projected Gradient Descent (PGD)

Given a convex function $f:\mathbb{R}^n \rightarrow \mathbb{R}$ and a convex set $\mathcal{C} \subseteq \mathbb{R}^n$, we would like to solve:
$$
\min_{x \in \mathcal{C}} f(x)
$$
where $\mathcal{C}$ represents constraints and can be written in the form of:
- $f_i(x) \leq 0 \forall i = 0, \cdots, N$
- $h_i(x) = 0 \forall i = 0, \cdots, M$

The Projected Gradient Descent (PGD) can be implemented to solve above problem by using the following steps:
1. Initial $x_0 \in \mathcal{C}$, step size $\alpha > 0$, max iterations $T$
1. Compute gradient: $g_k = \nabla f(x_k)$
1. Take gradient step: $x_{k+1/2} = x_k - \alpha g_k$
1. Project onto $\mathcal{C}$: $x_{k+1} = \mathcal{P}_\mathcal{C}(x_{k+1/2})$ if constaints are violated
1. Repeat from step 2 until convergence



## Example
Let us use the above algorithm to solve the following constrained optimization problem:
$$
\min_{x, y} f(x, y) = (x-1)^2 + (y-2)^2 + xy
$$
subject to:
$$2x+3y \geq 10 $$

In [ ]:
def f(x: float, y: float) -> float:
    """Computes the objective function value.

    Args:
        x (float): The x-coordinate.
        y (float): The y-coordinate.

    Returns:
        float: The computed function value.
    """
    return (x - 1)**2 + (y - 2)**2 + x * y

def gradient(x: float, y: float) -> np.ndarray:
    """Computes the gradient vector of the function f(x, y).

    Args:
        x (float): The x-coordinate.
        y (float): The y-coordinate.

    Returns:
        np.ndarray: The gradient vector [df/dx, df/dy].
    """
    return np.array([2 * (x - 1) + y, 2 * (y - 2) + x])

# Constraint: 2x + 3y >= 10 (a=2, b=3, c=10)
a, b, c = 2, 3, 10

def project_onto_constraint(x: float, y: float) -> tuple[float, float]:
    """Projects the point (x, y) onto the constraint hyperplane 2x + 3y = 10.

    Args:
        x (float): The x-coordinate.
        y (float): The y-coordinate.

    Returns:
        tuple[float, float]: The projected coordinates (x_proj, y_proj).
    """
    denom = a**2 + b**2
    x_proj = x - a * (a*x + b*y - c) / denom
    y_proj = y - b * (a*x + b*y - c) / denom
    return x_proj, y_proj

def projected_gradient_descent(max_iters: int, tol: float, alpha: float) -> tuple[float, float, list[tuple[float, float]]]:
    """Performs projected gradient descent with constraints.

    Args:
        max_iters (int): Maximum number of iterations.
        tol (float): Convergence tolerance.
        alpha (float): Step size for gradient descent.

    Returns:
        tuple[float, float, list[tuple[float, float]]]:
            - The optimal x-coordinate.
            - The optimal y-coordinate.
            - A list of all (x, y) points visited during optimization.
    """
    # Initialize with a feasible point
    x, y = 4.0, 5.0
    # Store all iterations
    history = [(x, y)]
    print(f"Iter 0: x = {x:.4f}, y = {y:.4f}, f(x,y) = {f(x, y):.4f}, Constraint: {2*x + 3*y >= 10}")

    for k in range(1, max_iters + 1):
        grad = gradient(x, y)
        x_temp = x - alpha * grad[0]
        y_temp = y - alpha * grad[1]

        if 2*x_temp + 3*y_temp < 10:
            x_new, y_new = project_onto_constraint(x_temp, y_temp)
        else:
            x_new, y_new = x_temp, y_temp

        history.append((x_new, y_new))

        if np.linalg.norm([x_new - x, y_new - y]) < tol:
            x, y = x_new, y_new
            print(f"Iter {k}: x = {x:.4f}, y = {y:.4f}, f(x,y) = {f(x, y):.4f}, Constraint: {2*x + 3*y >= 10}")
            print(f"Converged at iteration {k}")
            break

        x, y = x_new, y_new
        print(f"Iter {k}: x = {x:.4f}, y = {y:.4f}, f(x,y) = {f(x, y):.4f}, Constraint: {2*x + 3*y >= 10}")

    return x, y, history

# Run the algorithm
x_opt, y_opt, history = projected_gradient_descent(max_iters=1000, tol=1e-2, alpha=0.1)
print(f"\nOptimal solution: x* = {x_opt:.4f}, y* = {y_opt:.4f}")
print(f"Objective value: {f(x_opt, y_opt):.4f}")
print(f"Constraint satisfied: {2*x_opt + 3*y_opt >= 10}")
print(f"Constraint Value: {2*x_opt + 3*y_opt}")

Let us now plot the results.

In [ ]:
# Create contour plot
limit_low = 1
limit_up = 5
x_vals = np.linspace(-limit_low, limit_up, 100)
y_vals = np.linspace(-limit_low, limit_up, 100)
X, Y = np.meshgrid(x_vals, y_vals)
Z = f(X, Y)

plt.figure(figsize=(10, 6))

# Plot contours
contours = plt.contour(X, Y, Z, levels=20, cmap='viridis')
plt.colorbar(contours, label='Objective Value')

# Plot constraint boundary (2x + 3y = 10)
constraint_x = np.linspace(-limit_low, limit_up, 100)
constraint_y = (10 - 2 * constraint_x) / 3
plt.plot(constraint_x, constraint_y, 'r-', label='Constraint: 2x + 3y = 10')

# Shade feasible region (2x + 3y >= 10)
plt.fill_between(constraint_x, constraint_y, limit_up, color='red', alpha=0.1, label='Feasible Region')

# Plot PGD iterations
x_hist, y_hist = zip(*history)
plt.scatter(x_hist, y_hist, c='white', s=30, edgecolors='black', label='PGD Iterations')
plt.plot(x_hist, y_hist, 'k--', lw=1, alpha=0.5)

# Mark initial and optimal points
plt.scatter(x_hist[0], y_hist[0], c='blue', s=100, label='Initial Point', marker='s')
plt.scatter(x_opt, y_opt, c='red', s=100, label='Optimal Point', marker='*')

plt.xlabel('x')
plt.ylabel('y')
plt.title('Projected Gradient Descent Optimization')
plt.legend()
plt.grid(True)
plt.xlim(-limit_low, limit_up)
plt.ylim(-limit_low, limit_up)
plt.show()

## Exercise 1

Let us use the above algorithm to solve the following constrained optimization problem:
$$
\min_{x, y} f(x, y) = (x-1)^2 + (y-2)^2 + xy
$$
subject to:
$$ (x-5)^2 + (y-4)^2 \leq 8 $$

In [ ]:
def f(x: float, y: float) -> float:
    """Computes the objective function value.

    Args:
        x (float): The x-coordinate.
        y (float): The y-coordinate.

    Returns:
        float: The computed function value.
    """
    return (x - 1)**2 + (y - 2)**2 + x * y

def grad_x(x: float, y: float) -> float:
    """Computes the partial derivative of f with respect to x.

    Args:
        x (float): The x-coordinate.
        y (float): The y-coordinate.

    Returns:
        float: The gradient component df/dx.
    """
    return 2 * (x - 1) + y

def grad_y(x: float, y: float) -> float:
    """Computes the partial derivative of f with respect to y.

    Args:
        x (float): The x-coordinate.
        y (float): The y-coordinate.

    Returns:
        float: The gradient component df/dy.
    """
    return 2 * (y - 2) + x

radius: float = 8

def project_onto_circle(x: float, y: float, radius: float) -> tuple[float, float]:
    """Projects (x, y) onto the circle (x-5)^2 + (y-4)^2 <= 3.

    Args:
        x (float): The x-coordinate.
        y (float): The y-coordinate.
        radius (float): The radius of the constraint circle.

    Returns:
        tuple[float, float]: The projected coordinates (x_proj, y_proj).
    """
    center_x, center_y = 5, 4
    radius = np.sqrt(radius)

    # Compute distance from center
    dx = x - center_x
    dy = y - center_y
    distance = np.sqrt(dx**2 + dy**2)

    # Project onto boundary
    x_proj = # Your code
    y_proj = # Your code
    return x_proj, y_proj

def pgd_circle(max_iters: int, alpha: float, tol: float) -> tuple[float, float, list[tuple[float, float]]]:
    """Performs projected gradient descent constrained within a circle.

    Args:
        max_iters (int): Maximum number of iterations.
        alpha (float): Step size for gradient descent.
        tol (float): Convergence tolerance.

    Returns:
        tuple[float, float, list[tuple[float, float]]]:
            - The optimal x-coordinate.
            - The optimal y-coordinate.
            - A list of all (x, y) points visited during optimization.
    """
    x, y = 5.0, 3.0  # Initialize with an infeasible point outside the circle
    history = [(x, y)]

    for k in range(max_iters):
        # Gradient step
        x_temp = x - alpha * grad_x(x, y)
        y_temp = # Your code

        # Projection step
        if (x_temp - 5)**2 + (y_temp - 4)**2 <= radius:
            x_new, y_new = x_temp, y_temp
        else:
            x_new, y_new = project_onto_circle(x_temp, y_temp, radius)

        # Check convergence
        if np.linalg.norm([x_new - x, y_new - y]) < tol:
            x, y = x_new, y_new
            history.append((x, y))
            print(f"Iter {k}: x = {x:.4f}, y = {y:.4f}, f(x,y) = {f(x, y):.4f}, Constraint: {(x-5)**2 + (y-4)**2 <= radius}")
            print(f"Converged at iteration {k}")
            break

        x, y = x_new, y_new
        history.append((x, y))
        print(f"Iter {k}: x = {x:.4f}, y = {y:.4f}, f(x,y) = {f(x, y):.4f}, Constraint: {(x-5)**2 + (y-4)**2 <= radius}")

    return x, y, history

# Run PGD
x_opt, y_opt, history = pgd_circle(max_iters=1000, alpha=0.1, tol=1e-3)
print(f"Optimal solution: x* = {x_opt:.4f}, y* = {y_opt:.4f}")
print(f"Objective value: {f(x_opt, y_opt):.4f}")
print(f"Constraint check: {(x_opt-5)**2 + (y_opt-4)**2 <= 3}")
print(f"Constraint Value: {(x_opt-5)**2 + (y_opt-4)**2}")


In [ ]:
limit_low = -1
limit_up = 6
radius = 8

# Plot the circle constraint
theta = np.linspace(0, 2*np.pi, 100)
circle_x = 5 + np.sqrt(radius) * np.cos(theta)
circle_y = 4 + np.sqrt(radius) * np.sin(theta)

# Plot objective contours
x_vals = np.linspace(limit_low, limit_up, 100)
y_vals = np.linspace(limit_low, limit_up, 100)
X, Y = np.meshgrid(x_vals, y_vals)
Z = f(X, Y)

plt.figure(figsize=(10, 6))
plt.contour(X, Y, Z, levels=20, cmap='viridis')
plt.colorbar(label='Objective Value')
plt.plot(circle_x, circle_y, 'r-', label='Constraint Boundary')
plt.fill(circle_x, circle_y, color='red', alpha=0.1, label='Feasible Region')

# Plot PGD path
x_hist, y_hist = zip(*history)
plt.scatter(x_hist, y_hist, c='white', s=30, edgecolors='black')
plt.plot(x_hist, y_hist, 'k--', lw=1, alpha=0.5, label='PGD Path')

# Mark points
plt.scatter(x_hist[0], y_hist[0], c='blue', s=100, label='Initial Point', marker='s')
plt.scatter(x_opt, y_opt, c='red', s=100, label='Optimal Point', marker='*')

plt.xlabel('x')
plt.ylabel('y')
plt.title('PGD with Circle Constraint')
plt.legend()
plt.grid(True)
plt.xlim(limit_low, limit_up)
plt.ylim(limit_low, limit_up)
plt.show()

# Alternating Optimization
To solve a Biconvex problem:
1. Partition $\mathbf{x}=(X_1,X_2,\cdots,X_i)$ into sets of non-overlapping variables
1. Compute $$X_i^{(r+1)} = \textit{argmin/max}\ f(X_1^{(r+1)}, X_2^{(r+1)}, \cdots, X_i, \cdots, X_t^{(r)}),$$ with $i=1, \cdots, t$ where $r$ is the index of the current iteration.
1. If $||x^{(r+1)}-x^{(r)}|| \leq \epsilon $ or $r\geq L$, then quit; otherwise, set $r=r+1$ and go to step 2

## Example
Let us use the above algorithm to solve the following constrained optimization problem:
$$
\min_{x, y} f(x, y) = (x-1)^2 + (y-2)^2 + xy
$$
subject to:
$$x \geq 1$$
$$y \geq 3$$

In [ ]:
def f(x: float, y: float) -> float:
    """Computes the objective function value.

    Args:
        x (float): The x-coordinate.
        y (float): The y-coordinate.

    Returns:
        float: The computed function value.
    """
    return (x - 1)**2 + (y - 2)**2 + x * y

def grad_x(x: float, y: float) -> float:
    """Computes the partial derivative of f with respect to x.

    Args:
        x (float): The x-coordinate.
        y (float): The y-coordinate.

    Returns:
        float: The gradient component df/dx.
    """
    return 2 * (x - 1) + y

def grad_y(x: float, y: float) -> float:
    """Computes the partial derivative of f with respect to y.

    Args:
        x (float): The x-coordinate.
        y (float): The y-coordinate.

    Returns:
        float: The gradient component df/dy.
    """
    return 2 * (y - 2) + x

def ao_pgd(max_iters: int, tol: float, alpha_x: float, alpha_y: float) -> tuple[float, float, list[tuple[float, float]]]:
    """Performs alternating optimization projected gradient descent.

    Args:
        max_iters (int): Maximum number of iterations.
        tol (float): Convergence tolerance.
        alpha_x (float): Step size for x update.
        alpha_y (float): Step size for y update.

    Returns:
        tuple[float, float, list[tuple[float, float]]]:
            - The optimal x-coordinate.
            - The optimal y-coordinate.
            - A list of all (x, y) points visited during optimization.
    """
    # Initialize with a feasible point (x >= 1 and y >= 3)
    x, y = 5.0, 4.0
    # Store all iterations
    history = [(x, y)]
    print(f"Iter 0: x = {x:.4f}, y = {y:.4f}, f(x,y) = {f(x, y):.4f}, Constraint: {(x >= 1) and (y >= 3)}")

    for k in range(1, max_iters + 1):
        # Step 1: Fix y, update x
        x_temp = x - alpha_x * grad_x(x, y)

        # Project onto x >= 1
        x_new = max(x_temp, 1.0)

        # Step 2: Fix x_new, update y
        y_temp = y - alpha_y * grad_y(x_new, y)

        # Project onto y >= 3
        y_new = max(y_temp, 3.0)

        # Store the new point
        history.append((x_new, y_new))

        # Check convergence
        if np.linalg.norm([x_new - x, y_new - y]) < tol:
            x, y = x_new, y_new
            print(f"Iter {k}: x = {x:.4f}, y = {y:.4f}, f(x,y) = {f(x, y):.4f}, Constraint: {(x >= 1) and (y >= 3)}")
            print(f"Converged at iteration {k}")
            break

        x, y = x_new, y_new
        print(f"Iter {k}: x = {x:.4f}, y = {y:.4f}, f(x,y) = {f(x, y):.4f}, Constraint: {(x >= 1) and (y >= 3)}")

    return x, y, history

# Run the algorithm
x_opt, y_opt, history = ao_pgd(max_iters=1000, tol=1e-2, alpha_x=0.1, alpha_y=0.1)
print(f"\nOptimal solution: x* = {x_opt:.4f}, y* = {y_opt:.4f}, Constraint satisfied: {(x_opt >= 1) and (y_opt >= 3)}")


Let us now plot the results.

In [ ]:
# Create contour plot
limit_low = -1
limit_up = 6
x_vals = np.linspace(limit_low, limit_up, 100)
y_vals = np.linspace(limit_low, limit_up, 100)
X, Y = np.meshgrid(x_vals, y_vals)
Z = f(X, Y)

plt.figure(figsize=(10, 6))

# Plot contours
contours = plt.contour(X, Y, Z, levels=20, cmap='viridis')
plt.colorbar(contours, label='Objective Value')

# Plot constraint boundary
constraint_x = 1
constraint_y = 3
# Plot horizontal and vertical lines
plt.axhline(y=constraint_y, color='r', linestyle='--', label="y = 0")  # Horizontal line at y=0
plt.axvline(x=constraint_x, color='b', linestyle='-.', label="x = 0")  # Vertical line at x=0

# Shade feasible region (x > 1 and y > 3)
x_shade = [constraint_x, limit_up, limit_up, constraint_x]  # x-coordinates of shaded area
y_shade = [constraint_y, constraint_y, limit_up, limit_up]  # y-coordinates of shaded area
plt.fill(x_shade, y_shade, color='red', alpha=0.3, label='Feasible Region')

# Plot PGD iterations
x_hist, y_hist = zip(*history)
plt.scatter(x_hist, y_hist, c='white', s=30, edgecolors='black', label='PGD Iterations')
plt.plot(x_hist, y_hist, 'k--', lw=1, alpha=0.5)

# Mark initial and optimal points
plt.scatter(x_hist[0], y_hist[0], c='blue', s=100, label='Initial Point', marker='s')
plt.scatter(x_opt, y_opt, c='red', s=100, label='Optimal Point', marker='*')

plt.xlabel('x')
plt.ylabel('y')
plt.title('Projected Gradient Descent Optimization')
plt.legend()
plt.grid(True)
plt.xlim(limit_low, limit_up)
plt.ylim(limit_low, limit_up)
plt.show()

## Exercise 2

Let us use the above algorithm to solve the following constrained optimization problem:
$$
\min_{x, y} f(x, y) = (x-1)^2 + (y-2)^2 + xy
$$
subject to:
$$x^2 \geq 2$$
$$y \geq 3$$

In [ ]:
def f(x: float, y: float) -> float:
    """Computes the objective function value.

    Args:
        x (float): The x-coordinate.
        y (float): The y-coordinate.

    Returns:
        float: The computed function value.
    """
    return (x - 1)**2 + (y - 2)**2 + x * y

def grad_x(x: float, y: float) -> float:
    """Computes the partial derivative of f with respect to x.

    Args:
        x (float): The x-coordinate.
        y (float): The y-coordinate.

    Returns:
        float: The gradient component df/dx.
    """
    return 2 * (x - 1) + y

def grad_y(x: float, y: float) -> float:
    """Computes the partial derivative of f with respect to y.

    Args:
        x (float): The x-coordinate.
        y (float): The y-coordinate.

    Returns:
        float: The gradient component df/dy.
    """
    return 2 * (y - 2) + x

def project_x(x: float) -> float:
    """Projects x onto the constraint set {x | x² ≥ 2}.

    Args:
        x (float): The x-coordinate before projection.

    Returns:
        float: The projected x-coordinate.
    """
    return # Your code

def ao_pgd(max_iters: int, tol: float, alpha_x: float, alpha_y: float) -> tuple[float, float, list[tuple[float, float]]]:
    """Performs alternating optimization projected gradient descent.

    Args:
        max_iters (int): Maximum number of iterations.
        tol (float): Convergence tolerance.
        alpha_x (float): Step size for x update.
        alpha_y (float): Step size for y update.

    Returns:
        tuple[float, float, list[tuple[float, float]]]:
            - The optimal x-coordinate.
            - The optimal y-coordinate.
            - A list of all (x, y) points visited during optimization.
    """
    # Initialize with a feasible point (x² ≥ 2 and y ≥ 3)
    x, y = -5.0, 4.0
    # Store all iterations
    history = [(x, y)]
    print(f"Iter 0: x = {x:.4f}, y = {y:.4f}, f(x,y) = {f(x, y):.4f}, Constraint: {(x**2 >= 2) and (y >= 3)}")

    for k in range(1, max_iters + 1):

        # Step 1: Fix y, update x
        x_temp = x - alpha_x * grad_x(x, y)

        # Project onto x² ≥ 2
        x_new = project_x(x_temp) if x_temp**2 < 2 else x_temp

        # Step 2: Fix x_new, update y
        y_temp = y - alpha_y * grad_y(x_new, y)

        # Project onto y ≥ 3
        y_new = # Your code

        # Store the new point
        history.append((x_new, y_new))

        # Check convergence
        if np.linalg.norm([x_new - x, y_new - y]) < tol:
            x, y = x_new, y_new
            print(f"Iter {k}: x = {x:.4f}, y = {y:.4f}, f(x,y) = {f(x, y):.4f}, Constraint: {(x**2 >= 2) and (y >= 3)}")
            print(f"Converged at iteration {k}")
            break

        x, y = x_new, y_new
        print(f"Iter {k}: x = {x:.4f}, y = {y:.4f}, f(x,y) = {f(x, y):.4f}, Constraint: {(x**2 >= 2) and (y >= 3)}")

    return x, y, history

# Run the algorithm
x_opt, y_opt, history = ao_pgd(max_iters=1000, tol=1e-2, alpha_x=0.1, alpha_y=0.1)
print(f"\nOptimal solution: x* = {x_opt:.4f}, y* = {y_opt:.4f}, Constraint satisfied: {(x_opt**2 >= 2) and (y_opt >= 3)}")


In [ ]:
# Create contour plot
limit_low = -5
limit_up = 5
x_vals = np.linspace(limit_low, limit_up, 100)
y_vals = np.linspace(limit_low, limit_up, 100)
X, Y = np.meshgrid(x_vals, y_vals)
Z = f(X, Y)

plt.figure(figsize=(10, 6))

# Plot contours
contours = plt.contour(X, Y, Z, levels=20, cmap='viridis')
plt.colorbar(contours, label='Objective Value')

# Plot constraint boundaries
# 1. For x² ≥ 2 (two vertical lines at x = ±√2)
sqrt2 = np.sqrt(2)
plt.axvline(x=sqrt2, color='r', linestyle='--', label="x = √2")
plt.axvline(x=-sqrt2, color='r', linestyle='--', label="x = -√2")

# 2. For y ≥ 3 (horizontal line)
plt.axhline(y=3, color='b', linestyle='-.', label="y = 3")

# Shade feasible region (x² ≥ 2 AND y ≥ 3)
# Left feasible region (x ≤ -√2 and y ≥ 3)
x_left = np.linspace(limit_low, -sqrt2, 100)
y_top = np.linspace(3, limit_up, 100)
X_left, Y_top = np.meshgrid(x_left, y_top)
plt.fill_between(x_left, 3, limit_up, color='red', alpha=0.1)

# Right feasible region (x ≥ √2 and y ≥ 3)
x_right = np.linspace(sqrt2, limit_up, 100)
plt.fill_between(x_right, 3, limit_up, color='red', alpha=0.1, label='Feasible Region')

# Plot PGD iterations
x_hist, y_hist = zip(*history)
plt.scatter(x_hist, y_hist, c='white', s=30, edgecolors='black', label='PGD Iterations')
plt.plot(x_hist, y_hist, 'k--', lw=1, alpha=0.5)

# Mark initial and optimal points
plt.scatter(x_hist[0], y_hist[0], c='blue', s=100, label='Initial Point', marker='s')
plt.scatter(x_opt, y_opt, c='red', s=100, label='Optimal Point', marker='*')

plt.xlabel('x')
plt.ylabel('y')
plt.title('Projected Gradient Descent with x² ≥ 2 and y ≥ 3 Constraints')
plt.legend()
plt.grid(True)
plt.xlim(limit_low, limit_up)
plt.ylim(limit_low, limit_up)
plt.show()

Solve the above problem using simultanous PGD algorithm.

In [ ]:
def f(x: float, y: float) -> float:
    """Computes the objective function value.

    Args:
        x (float): The x-coordinate.
        y (float): The y-coordinate.

    Returns:
        float: The computed function value.
    """
    return (x - 1)**2 + (y - 2)**2 + x * y

def grad_x(x: float, y: float) -> float:
    """Computes the partial derivative of f with respect to x.

    Args:
        x (float): The x-coordinate.
        y (float): The y-coordinate.

    Returns:
        float: The gradient component df/dx.
    """
    return 2 * (x - 1) + y

def grad_y(x: float, y: float) -> float:
    """Computes the partial derivative of f with respect to y.

    Args:
        x (float): The x-coordinate.
        y (float): The y-coordinate.

    Returns:
        float: The gradient component df/dy.
    """
    return 2 * (y - 2) + x

def project_x(x: float) -> float:
    """Projects x onto the constraint set {x | x² ≥ 2}.

    Args:
        x (float): The x-coordinate before projection.

    Returns:
        float: The projected x-coordinate.
    """
    return # Your code

def simultaneous_pgd(max_iters: int, tol: float, alpha_x: float, alpha_y: float) -> tuple[float, float, list[tuple[float, float]]]:
    """Performs simultaneous projected gradient descent.

    Args:
        max_iters (int): Maximum number of iterations.
        tol (float): Convergence tolerance.
        alpha_x (float): Step size for x update.
        alpha_y (float): Step size for y update.

    Returns:
        tuple[float, float, list[tuple[float, float]]]:
            - The optimal x-coordinate.
            - The optimal y-coordinate.
            - A list of all (x, y) points visited during optimization.
    """
    # Initialize with a feasible point (x² ≥ 2 and y ≥ 3)
    x, y = -5.0, 4.0

    # Store all iterations
    history = [(x, y)]
    print(f"Iter 0: x = {x:.4f}, y = {y:.4f}, f(x,y) = {f(x, y):.4f}, Constraint: {(x**2 >= 2) and (y >= 3)}")

    for k in range(1, max_iters + 1):

        # Compute updates simultaneously
        x_temp = x - alpha_x * grad_x(x, y)
        y_temp = y - alpha_y * grad_y(x, y)

        # Project onto feasible set
        x_new = project_x(x_temp) if x_temp**2 < 2 else x_temp

        # Project onto y ≥ 3
        y_new = # Your code

        # Store the new point
        history.append((x_new, y_new))

        # Check convergence
        if np.linalg.norm([x_new - x, y_new - y]) < tol:
            print(f"Converged at iteration {k}")
            break

        x, y = x_new, y_new
        print(f"Iter {k}: x = {x:.4f}, y = {y:.4f}, f(x,y) = {f(x, y):.4f}, Constraint: {(x**2 >= 2) and (y >= 3)}")

    return x, y, history

# Run the algorithm
x_opt, y_opt, history = simultaneous_pgd(max_iters=1000, tol=1e-2, alpha_x=0.1, alpha_y=0.1)
print(f"\nOptimal solution: x* = {x_opt:.4f}, y* = {y_opt:.4f}, Constraint satisfied: {(x_opt**2 >= 2) and (y_opt >= 3)}")


In [ ]:
# Create contour plot
limit_low = -3
limit_up = 6
x_vals = np.linspace(limit_low, limit_up, 100)
y_vals = np.linspace(limit_low, limit_up, 100)
X, Y = np.meshgrid(x_vals, y_vals)
Z = f(X, Y)

plt.figure(figsize=(10, 6))

# Plot contours
contours = plt.contour(X, Y, Z, levels=20, cmap='viridis')
plt.colorbar(contours, label='Objective Value')

# Plot constraint boundaries
# 1. For x² ≥ 2 (two vertical lines at x = ±√2)
sqrt2 = np.sqrt(2)
plt.axvline(x=sqrt2, color='r', linestyle='--', label="x = √2")
plt.axvline(x=-sqrt2, color='r', linestyle='--', label="x = -√2")

# 2. For y ≥ 3 (horizontal line)
plt.axhline(y=3, color='b', linestyle='-.', label="y = 3")

# Shade feasible region (x² ≥ 2 AND y ≥ 3)
# Left feasible region (x ≤ -√2 and y ≥ 3)
x_left = np.linspace(limit_low, -sqrt2, 100)
y_top = np.linspace(3, limit_up, 100)
X_left, Y_top = np.meshgrid(x_left, y_top)
plt.fill_between(x_left, 3, limit_up, color='red', alpha=0.1)

# Right feasible region (x ≥ √2 and y ≥ 3)
x_right = np.linspace(sqrt2, limit_up, 100)
plt.fill_between(x_right, 3, limit_up, color='red', alpha=0.1, label='Feasible Region')

# Plot PGD iterations
x_hist, y_hist = zip(*history)
plt.scatter(x_hist, y_hist, c='white', s=30, edgecolors='black', label='PGD Iterations')
plt.plot(x_hist, y_hist, 'k--', lw=1, alpha=0.5)

# Mark initial and optimal points
plt.scatter(x_hist[0], y_hist[0], c='blue', s=100, label='Initial Point', marker='s')
plt.scatter(x_opt, y_opt, c='red', s=100, label='Optimal Point', marker='*')

plt.xlabel('x')
plt.ylabel('y')
plt.title('Projected Gradient Descent with x² ≥ 2 and y ≥ 3 Constraints')
plt.legend()
plt.grid(True)
plt.xlim(limit_low, limit_up)
plt.ylim(limit_low, limit_up)
plt.show()